In [111]:
import geopandas as gpd
import pandas as pd
import numpy as np



In [112]:
filename = "gadm36_AUT_2.shp"
letters = filename.split("_")[1]
regions_level2 = gpd.read_file(f"../country_boundaries/gadm36_AUT_shp/{filename}")
regions_level2.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,AUT,Austria,AUT.1_1,Burgenland,None,AUT.1.2_1,Eisenstadt,None,None,Statutarstadt,Statutory City,None,AT.BU.ES,"POLYGON ((16.58044 47.84060, 16.54771 47.82599..."
1,AUT,Austria,AUT.1_1,Burgenland,None,AUT.1.1_1,Eisenstadt Umgebung,None,None,Berzirk,District,None,AT.BU.EU,"POLYGON ((16.78435 47.88580, 16.76760 47.87045..."
2,AUT,Austria,AUT.1_1,Burgenland,None,AUT.1.3_1,Güssing,None,None,Berzirk,District,None,AT.BU.GS,"POLYGON ((16.29767 46.99884, 16.29373 47.00485..."
3,AUT,Austria,AUT.1_1,Burgenland,None,AUT.1.4_1,Jennersdorf,None,None,Berzirk,District,None,AT.BU.JE,"POLYGON ((16.06170 46.84864, 16.06170 46.84860..."
4,AUT,Austria,AUT.1_1,Burgenland,None,AUT.1.5_1,Mattersburg,None,None,Berzirk,District,None,AT.BU.MA,"POLYGON ((16.37243 47.64373, 16.35465 47.66730..."


In [113]:
# get regions names for each state
regions_level2 = regions_level2[['NAME_1', 'NAME_2', 'geometry']]
regions_level2.columns = ['state', 'region', 'geometry']
regions_level2.head()


,state,region,geometry
0,Burgenland,Eisenstadt,"POLYGON ((16.58044 47.84060, 16.54771 47.82599..."
1,Burgenland,Eisenstadt Umgebung,"POLYGON ((16.78435 47.88580, 16.76760 47.87045..."
2,Burgenland,Güssing,"POLYGON ((16.29767 46.99884, 16.29373 47.00485..."
3,Burgenland,Jennersdorf,"POLYGON ((16.06170 46.84864, 16.06170 46.84860..."
4,Burgenland,Mattersburg,"POLYGON ((16.37243 47.64373, 16.35465 47.66730..."


In [114]:
# create a list with the names of the regions for a each state
states = regions_level2.groupby('state')['region'].apply(list).reset_index()
states

,state,region
0,Burgenland,"[Eisenstadt, Eisenstadt Umgebung, Güssing, Jen..."
1,Kärnten,"[Feldkirchen, Hermagor, Klagenfurt, Klagenfurt..."
2,Niederösterreich,"[Amstetten, Baden, Bruck an der Leitha, Gänser..."
3,Oberösterreich,"[Braunau am Inn, Eferding, Freistadt, Gmunden,..."
4,Salzburg,"[Hallein, Salzburg, Salzburg Umgebung, Sankt J..."
5,Steiermark,"[Bruck-Mürzzuschlag, Deutschlandsberg, Graz, G..."
6,Tirol,"[Imst, Innsbruck, Innsbruck Land, Kitzbühel, K..."
7,Vorarlberg,"[Bludenz, Bodensee, Bregenz, Dornbirn, Feldkirch]"
8,Wien,[Wien]


In [115]:
# generate search queries for all types of farms in each regions of each state
farm_types = [
    "dairy farms",
    "poultry farms",
    "cattle farms",
    "livestock farms",
    "pig farms",
    "fish farms",
    "aquaculture farms",
    "egg farmers",
    "chicken hatchery",
    "shrimp farms",
    "seafood farms",
    "beef farms",
    "meat producer"
]

queries = []

for state in states['state']:
    for farm_type in farm_types:
        for region in states[states['state'] == state]['region'].values[0]:
        #for farm_type in farm_types:
            queries.append(f'{farm_type} in {region}, {state}')
            




In [116]:
#store the list in a pandas dataframe
queries_df = pd.DataFrame(queries, columns=['query'])
# divide the dataframe into each state based on the list of states
queries_df2 = queries_df.copy()
queries_df2['state'] = queries_df['query'].apply(lambda x: x.split(', ')[1])

queries_df2.state.unique()

array(['Burgenland', 'Kärnten', 'Niederösterreich', 'Oberösterreich',
       'Salzburg', 'Steiermark', 'Tirol', 'Vorarlberg', 'Wien'],
      dtype=object)

In [117]:
queries_df3 = queries_df2.copy()
queries_df3.drop(columns=['state']).to_csv(f'farm_queries_{letters}.csv', index=False)

In [118]:
list_states = list(queries_df2.state.unique())

list_states

['Burgenland',
 'Kärnten',
 'Niederösterreich',
 'Oberösterreich',
 'Salzburg',
 'Steiermark',
 'Tirol',
 'Vorarlberg',
 'Wien']

In [119]:
import re

#queries_df3 = queries_df2.copy()

# Loop through each state
#for state in list_states:
    # Replace special characters and spaces with underscores
   # state_file_name = re.sub(r'[^a-zA-Z0-9]', '_', state)

    # Filter DataFrame based on state
   # state_data = queries_df3[queries_df3['state'] == state]

     #state_data.to_csv(f'farm_queries_{state_file_name}.csv', index=False)
    # Save only the queries for that state to a CSV file
   # state_data.drop(columns=['state']).to_csv(f'farm_queries_{state_file_name}.csv', index=False)


In [120]:
df = pd.read_csv(f'farm_queries_{letters}.csv')

# Generate queries dynamically from DataFrame to list format for web scraping
queries4 = []
for _, row in df.iterrows():
    query = f"{row['query']}"
    queries4.append(query)
    
queries4

['dairy farms in Eisenstadt, Burgenland',
 'dairy farms in Eisenstadt Umgebung, Burgenland',
 'dairy farms in Güssing, Burgenland',
 'dairy farms in Jennersdorf, Burgenland',
 'dairy farms in Mattersburg, Burgenland',
 'dairy farms in Neusiedl am See, Burgenland',
 'dairy farms in Oberpullendorf, Burgenland',
 'dairy farms in Oberwart, Burgenland',
 'dairy farms in Rust, Burgenland',
 'poultry farms in Eisenstadt, Burgenland',
 'poultry farms in Eisenstadt Umgebung, Burgenland',
 'poultry farms in Güssing, Burgenland',
 'poultry farms in Jennersdorf, Burgenland',
 'poultry farms in Mattersburg, Burgenland',
 'poultry farms in Neusiedl am See, Burgenland',
 'poultry farms in Oberpullendorf, Burgenland',
 'poultry farms in Oberwart, Burgenland',
 'poultry farms in Rust, Burgenland',
 'cattle farms in Eisenstadt, Burgenland',
 'cattle farms in Eisenstadt Umgebung, Burgenland',
 'cattle farms in Güssing, Burgenland',
 'cattle farms in Jennersdorf, Burgenland',
 'cattle farms in Mattersburg